<div class="notebook-buttons" style="display:flex; padding-top: 5rem;padding-bottom: 2.5rem;line-height: 2.15;">
    <a href="https://colab.research.google.com/github/magdasalatka/fantastic-features/blob/main/main.ipynb">
        <div id="colab-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; align-items: center;">
            <img class="call-to-action-img" src="img/colab.svg" width="30" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Run in Google Colab</div>
        </div>
    </a>
    <a href="https://raw.githubusercontent.com/magdasalatka/fantastic-features/main/main.ipynb" download>
        <div id="download-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/download.svg" width="22" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Download Notebook</div>
        </div>
    </a>
    <a href="https://github.com/magdasalatka/fantastic-features/blob/main/main.ipynb">
        <div id="github-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/github.svg" width="25" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">View on GitHub</div>
        </div>
    </a>
</div>

# Back to the Feature
### boost you model with statistical feature engineering
by [Teresa Kubacka](http://teresa-kubacka.com/), [Magdalena Surówka](https://datali.ch)

In [ ]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import statsmodels.api as sm
import statsmodels.graphics as sg
import math

from dataset_noise_generator import *
from plotting import plot_x_vs_y, plot_x_over_time, plot_fitted_vs_residuals, calculate_variances, plot_residuals_distribution


# 1. Model diagnostics

Regression has some assumptions about its errors, *e<sub>i</sub>*. 
These assumptions are:

 * E[*e<sub>i</sub>*] = 0
 * Var(*e<sub>i</sub>*) = &sigma;<sup>2</sup>
 * *e<sub>i</sub>* ~ *N*(0, &sigma;<sup>2</sup>)
 * Cov(*e<sub>i</sub>*, *e<sub>j</sub>*) = 0

 Additioanlly, we also assume:
 * no multicollinearity in regressors

The first three conditions are necessary to perform a least square estimation and to have valid fitted values.  
The last condition is only required for any hypothesis tests, confidence intervals and prediction intervals. 

As you can see, most of our assumptions are about errors. However, errors cannot be observed in practice. Instead, we well be working with residuals, which are only estimates of the errors. They have however, an estimation-related issue.   
In regression, the variances of the residuals at different input variable values may differ. This can happen even if the variances of the errors at these different input variable values are equal.   

To improve the results, we can standardize or [studentize](https://en.wikipedia.org/wiki/Studentized_residual) the residuals.

## 1.1 Sample model
Let's fit a sample model, and take a look at its fit.

In [ ]:
df = pd.read_csv("data/interesting_data.csv", index_col=0)
y = df.y.to_numpy()
X = df.loc[:, df.columns != 'y'].to_numpy()

threshold = int(len(X)*0.8)
X_train, X_test = X[:threshold], X[threshold:]
y_train, y_test = y[:threshold], y[threshold:]

predictors = sm.add_constant(X_train)
model = sm.OLS(y_train, predictors).fit()
fitted = model.predict(predictors)

print(model.summary())

## 1.2 Assumption 1
E[*e<sub>i</sub>*] = 0

In other words, on average the errrors should be zero

In [ ]:
residuals_raw = fitted - y_train
residuals = residuals_raw/np.std(residuals_raw)
print("Expected error estimate: {}".format(sum(residuals)/len(residuals)))

In [ ]:
plot_x_vs_y(fitted, residuals, "response", "residuals")

In [ ]:
plot_x_vs_y(fitted, residuals, "Fitted", "Residuals", add_zero_line=True, add_lowess=True)

Failing assumption implies systematic error. This means:
* the relationship between response and regressors may be nonlinear
* some important regressors may be missing
* some important interactions may be missing

## 1.3 Assumption 2

Var(*e<sub>i</sub>*) = &sigma;<sup>2</sup>

In other words: variance of residuals should be constant

### 1.3.1 Fitted vs residuals: heteroskedasticity check

In [ ]:
# Let's extend our plot_fitted_vs_residuals function
def plot_fitted_vs_residuals(fitted: np.ndarray, residuals: np.ndarray, 
                             mean: bool=True, 
                             width = 100, 
                          bin_type = None) -> None:

    ax = plot_x_vs_y(fitted, residuals, "Fitted", "Residuals")
    
    if mean:
        ax.hlines(0, xmin=min(fitted), xmax=max(fitted), colors="orange")
    if bin_type == "window":
        ax.vlines(np.arange(min(fitted), max(fitted)+width, width), 
                  ymin=min(residuals), ymax=max(residuals), colors="red")
    elif bin_type == "points": 
        width = round(width)
        ax.vlines(np.sort(fitted)[::width], 
                      ymin=min(residuals), ymax=max(residuals), colors="red")
    plt.title('Fitted vs Residuals')

plot_fitted_vs_residuals(fitted, residuals, mean=False, width=100, bin_type='window')

### 1.3.2 Residuals: heteroskedasticity check

In [ ]:
# TODO: Try different widths
bins, variances, counts = calculate_variances(fitted, residuals, 100, bin_type="points")

plt.bar(bins, variances, width=np.diff(bins).min()*0.8, align='edge')
plt.title('Variances')
plt.show()


Failing assumption implies heteroskedasticity. This means:
* the error estimates are not valid =>
* the confidence intervals are not valid =>
* p-values are not valid
* coefficients **are valid**  

In short: your expected value remains unchanged. But you have no viable insights on model unsertanity.

## 1.4 Assumption 3
*e<sub>i</sub>* ~ *N*(0, &sigma;<sup>2</sup>)

In other words: residuals are normally distributed

### 1.4.1 Empirical distribution

In [ ]:
plot_residuals_distribution(residuals, bins=40)

### 1.4.2 QQ-plot

In [ ]:
fig = sm.qqplot(residuals/np.std(residuals), line='45')

Failing assumption implies systematic deviation. This means:
* model is failing to capture certain range of values
* model structure is not correct

In practice:
* few data points that are slightly "off the line" near the ends common, and usually tolerable
* skewed residuals need correction
* long-tailed, but symmetrical residuals are can be tolerable

## 1.5 Assumption 4
Cov(*e<sub>i</sub>*, *e<sub>j</sub>*) = 0

In other words: errors are not correlated **WITH EACH OTHER**


### 1.5.1 Residuals over time

In [ ]:
plot_x_over_time(residuals, "Residuals")

### 1.5.2 ACF plot
Check residuals vs lagged residuals

In [ ]:
fig = sg.tsaplots.plot_acf(residuals)

Failing assumption means:
* estimates are unbiased => expected value for coefficients and predictions is ok
* the estimate is not efficient => there are better regression models 
* standard errors are biased => confidence intervals, test statistics, and p-values are flawed


## 1.6 Assumption 5 (Optional)
No multicollinearlty

Regression does not have a unique solution if regressors are exactly linearly dependent. Often, we will find not perfect, but a strong correlation between variables. Multicollinearity means that there is such strong, yet not perfect, relation between the columns of X.

Under multicollinity, unique solution exists. However, it performs poorly in practice. 

### 1.6.1 Correlation plot

In [ ]:
def plot_correlations(X: np.ndarray) -> None:
    df = pd.DataFrame(X)

    f = plt.figure(figsize=(8, 8))
    plt.matshow(df.corr(), fignum=f.number)
    plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=45)
    plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title('Correlation Matrix', fontsize=16);

plot_correlations(X_train)

Multicollinearlity means:
* estimated coefficients have large standard errors
* the coefficients are imprecise
* it happens that none of the regressors is significant!
* small change in data can result in big change in results

# Exercise 1: Model diagnostics
@Teresa: Anything here? 

## 1. Load data 

Option 1: load predefined dataset: 

In [ ]:
X, y = datasets.load_diabetes(return_X_y=True)

print("X shape: {}".format(X.shape))
print("y shape: {}".format(y.shape))

Option 2: work with a synthetic dataset: 
- sklearn's `make_regression`
- modified version of `make_regression` that takes coefficients as an input 
- your own generated data (e.g. containing coupled variables, polynomial/non-linear relations etc.)

In [ ]:
num_indep = 3
n_sample = 1000

X_base, y_base, coeffs = make_regression_custom(n_samples=n_sample, n_features=num_indep, n_informative=num_indep,
                       tail_strength=0, bias=0, n_targets=1, noise=0, 
                           shuffle=False, coef=True, random_state=2, custom_coef=[30,15,20])
# noise = gen_noise(X_base, 0.1, 'sin', n_osc=3)+gen_noise(X_base, 0.1, 'normal')
# X = X_base + noise
X = X_base

# noise_y = gen_noise(y_base, 15, 'normal')*gen_noise(y_base, 0.1, 'linear_inc')
# y = y_base + noise_y


In [ ]:
yn = gen_noise(y_base,30,noise_type="poisson")*gen_noise(y_base,1,noise_type="normal")

y = y_base+yn

plt.plot(y_base,yn,marker='.',lw=0)

In [ ]:
yn = gen_noise(y_base,10,noise_type="linear_inc_index")*gen_noise(y_base,1,noise_type="normal")

y = y_base+yn

plt.plot(y_base,yn,marker='.',lw=0)

In [ ]:
yn = gen_noise(y_base,1,noise_type="normal_prop")

y = y_base+yn

plt.plot(y_base,yn,marker='.',lw=0)

In [ ]:
yn = gen_noise(y_base,1,noise_type="linear_inc_index", func=lambda x: np.log(np.abs(x)))*gen_noise(y_base,1,noise_type="normal")
# *gen_noise(y_base,1,noise_type="normal")

y = y_base+yn

plt.plot(y_base,yn,marker='.',lw=0)

In [ ]:
np.isneginf(yn).sum()

In [ ]:
yn = gen_noise(y_base,1,noise_type="linear_inc_index", func=lambda x: 1/np.sqrt(x))*gen_noise(y_base,1,noise_type="normal")
# *gen_noise(y_base,1,noise_type="normal")

y = y_base+yn

plt.plot(y_base,yn,marker='.',lw=0)

In [ ]:
y_base.shape

Train-test split

In [ ]:
threshold = int(len(X)*0.8)
X_train, X_test = X[:threshold], X[threshold:]
y_train, y_test = y[:threshold], y[threshold:]

## 2. Fit regression

## 2.1 Inspect data
How does the data look like?

In [ ]:
def plot_x_vs_y(x: np.ndarray, y: np.ndarray, x_name: str = 'x_name', y_name: str = 'y_name', 
                         add_zero_line: bool = False, add_lowess: bool = False, 
                         model = None, hexbin: bool = False, 
                         figsize = None, ax = None, lowess_param = 0.2, plot_args = {}) -> None:
    if ax is None: 
        _, axs = plt.subplots(1, 1, figsize=figsize, constrained_layout=True)
        this_ax = axs
    else:
        this_ax = ax
    
    if hexbin: 
        this_ax.hexbin(x, y, cmap="Greys", gridsize=40, **plot_args)
    else: 
        scatter_defaults = {'alpha': 0.25, 's': 20, 'edgecolors':None}
        for k,v in scatter_defaults.items():
            if k not in plot_args.keys():
                plot_args[k] = v        
        this_ax.scatter(x, y, lw=0, **plot_args)

    xlabel = x_name
    this_ax.set_xlabel(xlabel)
    this_ax.set_ylabel(y_name)
    this_ax.grid(True)
    if add_zero_line:
        this_ax.hlines(0, xmin=min(x), xmax=max(x), colors="magenta")
    if add_lowess: 
        sorted_xy = np.array(sorted(zip(x,y))).T
        sorted_x = sorted_xy[0,:]
        sorted_y = sorted_xy[1,:]
        smoothed = sm.nonparametric.lowess(exog=sorted_x, endog=sorted_y, frac=lowess_param)
#         print(smoothed.shape)
        this_ax.plot(smoothed[:,0],smoothed[:,1],lw=1,color='k')


    return this_ax

plot_x_vs_y(X_train[:,0], y_train, "Independent variable", "Dependent variable", 
                     add_zero_line=False, add_lowess=True,hexbin=False);

In [ ]:
plot_x_vs_y(X_train[:,0], y_train, "Independent variable", "Dependent variable", 
                     add_zero_line=True, hexbin=False, plot_args={'s':150, 'alpha':0.1});

In [ ]:
def plot_multiple_x_vs_y(X: np.ndarray, y: np.ndarray, x_name: str, y_name: str, 
                         add_regression_line: bool = False, add_zero_line: bool =False, 
                         model = None, hexbin: bool = False, n_col:int = 2,
                         figsize = None, plot_args = {}) -> None:

    if add_regression_line and (model is None):
        predictors = sm.add_constant(X)
        model = sm.OLS(y_train, predictors).fit()

    if len(X.shape)>1:
        n_vars = X.shape[1]
    else: 
        n_vars = 1
        X = X.reshape((len(X),1)) # otherwise X.T will enumerate the list element-wise
    
    n_row = math.ceil(n_vars/n_col)
    if figsize is None: 
        subplot_size = (3,3)
        figsize = (n_col*subplot_size[0], n_row*subplot_size[1])
    _, axs = plt.subplots(n_row, n_col, figsize=figsize, constrained_layout=True)

    for i, x in enumerate(X.T):
        row = i//n_col
        col = i%n_col
        
        if (n_vars == 1) and (n_col == 1) : 
            this_ax = axs
        elif n_row == 1: 
            this_ax = axs[col]
        else: 
            this_ax = axs[row,col]

        if n_vars > 1: 
            xlabel = "{} {}".format(x_name, i)
        else: 
            xlabel = x_name
            
        plot_x_vs_y(x=x, y=y, x_name=xlabel, y_name=y_name, 
                         add_zero_line= add_zero_line, 
                         model = model, hexbin = hexbin, 
                         figsize = figsize, ax = this_ax, plot_args=plot_args)     

        if add_regression_line and (model is not None):
            x_vals = np.array(this_ax.get_xlim())
            y_vals = model.params[0] + model.params[i+1] * x_vals
            this_ax.plot(x_vals, y_vals, '--', color="orange")
        
    return axs

plot_multiple_x_vs_y(X_train, y_train, "Independent variable", "Dependent variable", 
                     add_regression_line=True, hexbin=False, n_col=3, plot_args={'s':20});

## 2.2 Fitting the model

In [ ]:
# Scikit-learn version => No summary table
#lm = linear_model.LinearRegression()
#model = lm.fit(X_train, y_train)
#fitted = model.predict(X_train)

predictors = sm.add_constant(X_train)
model = sm.OLS(y_train, predictors).fit()
fitted = model.predict(predictors)

## 2.3 Analysing model output

In [ ]:
print(model.summary())

Working with Statsmodels output table: 

In [ ]:
model.pvalues

In [ ]:
model.params

In [ ]:
# compare with our coeffs: 
coeffs

In [ ]:
# export the table to pandas dataframe using .summary2(); _.tables is a tuple of 3 sub-tables, each of them is a df 
model.summary2().tables

In [ ]:
# .summary2() not always stable, a workaround is below, but then you lose some digits after comma:  
pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]

## 2.4 Actual vs predicted

In [ ]:
tmp_ax = plot_x_vs_y(fitted, y_train, "Predicted", "Observed", figsize=(6,6))
tmp_ax.plot([-150,150],[-150,150],lw=1,color='orange',zorder=0)

# Exercise 2:  Transformations

## Transformations

The idea behing data trasformations is to achieve a mean function that is linear in the transformed scale.

The most commonly used methods to transform variables are:
* Logarithmic transformation - np.log(X)
* Reciprocal transformation - 1 / X
* Square root transformation - X**(1/2)
* Exponential transformation (more general, you can use any exponent)
* Box-Cox transformation


## Exercise

In this exercise, we will practice variables transformation. 

First, load datasets from `data/ex1`.  
Then, for each dataset:  
* fit linear model: y = f(x) + e  
* calculate residuals  
* perform model diagnostics  
* try to improve your model using data transformations  

In [ ]:
# Load data
df = pd.read_csv("data/ex2/ds_1.csv")

# Your code


# Exercise 3: Fit your best model

Now, we will wrap it all together. Let's try to fit our best model to the dataset about rental prices.  
The data is available in `data/rent.csv`.

Procedure:
* Maka a model hypothesis
* Fit a model
* Run model diagnostics
* Start again

In [ ]:
# Your code
